# Resnet152 model for Covid-19 classification

### Import packages

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from PIL import Image
import os
import matplotlib.pyplot as plt

from tqdm import tqdm
import time
import datetime


In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round(elapsed, 2))
    return str(datetime.timedelta(seconds=elapsed_rounded))

### Dataset Parser

In [ ]:
class COVIDDataset(Dataset):
    def __init__(self, data_file, transform=None):
        self.data = []
        self.transform = transform
        self.map = {'negative': 0, 'positive': 1}
        
        with open(data_file, 'r') as f:
            folder = os.path.splitext(data_file)[0]
            for line in f:
                patient_id, filename, label, data_source = line.strip().split(' ')
                image_path = os.path.join(folder, filename)
                self.data.append((image_path ,self.map[label]))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image_path, label = self.data[index]
        image = Image.open(image_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

### Create Model

In [ ]:
# Define the modified ResNet152 model
class ModifiedResNet152(nn.Module):
    def __init__(self, num_classes=2, pretrained=True):
        super(ModifiedResNet152, self).__init__()
        if pretrained:
            self.resnet = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
        else:
            self.resnet = models.resnet152()
        self.resnet.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1024),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=1024, out_features=num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.resnet(x)
        return x
    

def train(model, train_loader, optimizer, criterion, device, mse=False):
    model.train()
    train_loss = 0.
    correct = 0.
    total = 0.
    print(f'Loading # {len(train_loader)} datas')
    
    with tqdm(train_loader, unit='batch') as tepoch:
        for data, target in tepoch:
            tepoch.set_description(f"Training")
            data, target = data.to(device), target.to(device)

            # instead of  optimizer.zero_grad(), do it in forloop for more efficiency
            for param in model.parameters():
                param.grad = None

            output = model(data)
            if not mse:
                loss = criterion(output, target)
            else:
                target_one_hot = torch.zeros(target.shape[0], 2).to(device)
                target_one_hot.scatter_(1, target.unsqueeze(1), 1)
                loss = criterion(output, target_one_hot.to(device))
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
        
    acc = 100.*correct/total
    return train_loss, acc

def validate(model, val_loader, criterion, device, mse=False):
    model.eval()
    val_loss = 0.
    correct = 0.
    total = 0.
    false_positive = 0
    false_negative = 0
    true_positive = 0
    true_negative = 0
    
    with torch.no_grad():
        with tqdm(val_loader, unit='batch') as tepoch:
            for data, target in tepoch:
                tepoch.set_description(f"Validating")
                data, target = data.to(device), target.to(device)
                output = model(data)
                if not mse:
                    val_loss += criterion(output, target).item()
                else:
                    target_one_hot = torch.zeros(target.shape[0], 2).to(device)
                    target_one_hot.scatter_(1, target.unsqueeze(1), 1)
                    val_loss += criterion(output, target_one_hot.to(device))
                _, predicted = output.max(1)
                total += target.size(0)
                false_positive += ((predicted == 1) & (target == 0)).sum().item()
                false_negative += ((predicted == 0) & (target == 1)).sum().item()
                true_negative += ((predicted == 0) & (target == 0)).sum().item()
                true_positive += ((predicted == 1) & (target == 1)).sum().item()
    correct = true_positive + true_negative
    sensitivity = true_positive / (true_positive + false_negative)
    specificity = true_negative / (true_negative + false_positive)
    f1_score = 2 * true_positive / (2 * true_positive + false_positive + false_negative)
    print(f'Validation loss: {val_loss:.4f}, Accuracy: {correct}/{total} ({100.*correct/total:.2f}%), Sensitivity: {sensitivity:.2f}, Specificity: {specificity:.2f}, F1 score: {f1_score:.2f}')
    acc = 100.*correct/total
    return val_loss, acc, sensitivity, specificity, f1_score


In [ ]:
def train_resnet(model, dataset, lr, epochs, batch_size, factor, patience, criterion, device, pretrain, state_for='unprovided', mse=False):
    """Train the model on the dataset and return the best accuracy
    
    Args:
        model: the model to train
        dataset: the dataset to train on
        lr: learning rate
        epochs: number of epochs to train for
        batch_size: batch size
        factor: factor for ReduceLROnPlateau
        patience: patience for ReduceLROnPlateau
        criterion: loss function
        device: device to train on (cuda or cpu)
        
    Returns:
        best_acc: best accuracy achieved
    """
    PRETRAIN = 'finetune' if pretrain else 'wo_pretrain'
    print(f"Using device: {device}")
    # get size of dataset
    dataset_size = len(dataset)
    print(f"Dataset size: {dataset_size}")
    # split dataset into train and validation sets
    train_size = int(0.8 * dataset_size)
    val_size = dataset_size - train_size
    train_dataset, val_dataset = data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,
                            shuffle=False)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=factor, patience=patience, verbose=True)
    # record training and validation losses and accuracies
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    best_acc = 0
    best_epoch = 0
    model_state_path = ""

    start = time.time()
    print(f"Began training for {epochs} epochs")
    for epoch in range(epochs):
        print(f'Training (Epoch {epoch + 1})')
        torch.cuda.empty_cache()
        t0 = time.time()
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device, mse)
        print("  * Training epoch took: {:}".format(format_time(time.time() - t0)))
        
        t0 = time.time()
        val_loss, val_acc, val_sen, val_spec, val_f1 = validate(model, val_loader, criterion, device, mse)
        print("  * Validation took: {:}".format(format_time(time.time() - t0)))

        # record training and validation losses and accuracies
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        scheduler.step(val_loss)

        print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.3f}\nVal Sensitivity: {val_sen:.3f}, Val Specificity: {val_spec:.3f}, Val F1 score: {val_f1:.3f}')
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_epoch = epoch + 1
            model_state_path = f'{state_for}_{PRETRAIN}_resnet152.pt'
            torch.save(model.state_dict(), model_state_path)
    
    print("Training complete in {}".format(format_time(time.time() - start)))
    print("Best validation accuracy: {:.2f}% -- Acquired with epoch {}".format(best_acc, best_epoch))

    return train_losses, val_losses, train_accs, val_accs, model_state_path

### Begin Training process for RESNET152

In [ ]:
# Set up hyperparameters for the model
# Set up hyperparameters and data loaders
# TUNABLE:
lr = 2e-4
# TUNABLE:
epochs = 10
# TUNABLE: original 64
batch_size = 16
# TUNABLE:
factor = 0.7
# TUNABLE:
patience = 5
# set the model with pretrained weight or not
pretrain = True 

num_classes = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # Define the transformations to be applied to the data
target_size = 224
transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize(target_size),
        transforms.CenterCrop(target_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
# create COVIDDataset object
# TUNABLE: --> can try different datasets
dataset = COVIDDataset('train.txt', transform=transform)

# Define the model, loss function
model = ModifiedResNet152(num_classes, pretrained=pretrain).to(device) # pretrained=True

# TUNABLE: Choose the loss function and optimizer.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
state_for = 'ADAM+CE'

# Remember to add a parameter to the train function to indicate whether to use MSE or not
# criterion = nn.functional.mse_loss
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# state_for = 'ADAM+MSE'

# criterion = nn.NLLLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# state_for = 'ADAM+NLL'

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# state_for = 'SGD'

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
# state_for = 'RMSPROP'

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, rho=0.9, eps=1e-06, weight_decay=0)
# state_for = 'ADADelta'

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# state_for = 'AdamW'



In [ ]:
# Train the model
train_losses, val_losses, train_accs, val_accs, model_state_path = train_resnet(model, dataset, lr, epochs, batch_size, factor, patience, criterion, device, pretrain, state_for)

### Plot the loss and accuracy

In [ ]:
# plot training and validation losses and accuracies
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d_%H%M%S")


plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Loss (ResNet152)")
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.title("Accuracy (ResNet152)")
plt.plot(train_accs, label='Training accuracy')
plt.plot(val_accs, label='Validation accuracy')
plt.legend()
plt.savefig('./resnet_plots/' + f'{state_for}_loss_resnet152_{now}.png')
plt.show()

### Test the resnet model

In [ ]:
# test data
test_dataset = COVIDDataset('test.txt', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model_path = model_state_path
# model_path = 'RMSPROP_finetune_resnet152.pt'

# load best model
model.load_state_dict(torch.load(model_path))
print(f'Model {model_path} loaded')
# evaluate model on test set
test_loss, test_acc, test_sen, test_spec, test_f1 = validate(model, test_loader, criterion, device)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.3f}%, Test sensitivity: {test_sen:.3f}%, Test specificity: {test_spec:.3f}%, Test F1 score: {test_f1:.3f}%')